# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of the project. 

#### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Required Queries

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
# Creates a map from CSV file column names to CSV indexes
fields = {"artist": 0, "firstName": 1, "gender": 2, "itemInSession": 3, "lastName": 4,
    "length": 5, "level": 6, "location": 7, "sessionId": 8,"song": 9, "userId": 10}

## Implementation of first query

__Goal__: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

__Primary Key Reasoning:__

The primary key is composed by `session_id` and `item_in_session` for the following reasons:

1. The `session_id` is used as partition key. This helps the database to distribute the data uniformely in the different nodes of the cluster.
2. The combination of both fields creates a unique identifier, just using `session` for primary key would mean losing data during the import operation.
3. The query requires both fields in the `where` clause and this forces us to include both fields in the primary key.

In [11]:
query = """
CREATE TABLE IF NOT EXISTS music_history
(session_id int, item_in_session int, artist text, song text, length double,
    PRIMARY KEY(session_id, item_in_session))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO music_history(session_id, item_in_session, artist, song, length) 
        VALUES(%s, %s, %s, %s, %s)
        """
        elements = (int(line[fields["sessionId"]]), int(line[fields["itemInSession"]]), 
                    line[fields["artist"]], line[fields["song"]], 
                    float(line[fields["length"]]))
        session.execute(query, elements)

__Solution to the first question__

In [13]:
# Select statement to solve the first question
query = """
SELECT artist, song, length
FROM music_history
WHERE session_id = 338 and item_in_session = 4
"""
try:
    rows = session.execute(query)
    for row in rows:
        print(row.artist, row.song, row.length)
except Exception as e:
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Implementation of second query

__Goal:__ Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

__Primary Key Reasoning:__

The primary key is composed by `user_id`, `session_id`, `item_in_session`, for the following reasons:

1. The combination of `session_id`, `item_in_session` uniquely identifies each record of the table, removing any of those fields would cause lost information in the ETL process.
2. `item_in_session` as a clustering column allows to sort the data based on that field. This is required by the problem statement.
3. `user_id` as a partition key column helps to uniformely distribute the data across nodes.

In [14]:
query = """
CREATE TABLE IF NOT EXISTS music_history_by_session
(user_id int, session_id int, item_in_session int, artist text, song text, user text, 
PRIMARY KEY(user_id, session_id, item_in_session))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO music_history_by_session(user_id, session_id, item_in_session, artist, song, user) 
        VALUES(%s, %s, %s, %s, %s, %s)
        """
        # Concatenates first and last name to create the user field
        user = line[fields["firstName"]] + ' ' + line[fields["lastName"]]
        elements = (int(line[fields["userId"]]), int(line[fields["sessionId"]]), 
                   int(line[fields["itemInSession"]]), line[fields["artist"]],
                    line[fields["song"]], user)
        session.execute(query, elements)

__Solution to the second question__

In [16]:
# Select statement to solve the second question
query = """
SELECT artist, song, user
FROM music_history_by_session 
WHERE user_id = 10 and session_id = 182
"""
try:
    rows = session.execute(query)
    for row in rows:
        print(row.artist, row.song, row.user)
except Exception as e:
    print(e)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Implementation of third query

__Goal:__

Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

__Primary Key Reasoning:__

The primary key is composed of `song`, `session_id`, `item_in_session` for the following reasons:

- `session_id`, `item_in_session` uniquely identify each record in the table.
- `song` is required in the `where` clause of the query.
- `song` as the clustering column helps to uniformely distribute data across nodes. It also speeds up the query (node where information is stored is known with the query)

In [17]:
query = """
CREATE TABLE IF NOT EXISTS users_by_song
(song text, session_id int, item_in_session int, user text, 
    PRIMARY KEY(song, session_id, item_in_session))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song(song, session_id, item_in_session, user) VALUES(%s, %s, %s, %s)"
        # Concatenates first and last name to create the user field
        user = line[fields["firstName"]] + ' ' + line[fields["lastName"]]
        elements = (line[fields["song"]], int(line[fields["sessionId"]]), 
                    int(line[fields["itemInSession"]]), user)
        session.execute(query, elements)

__Solution to the third question__

In [19]:
# Select statement to solve the third question
query = """
SELECT user
FROM users_by_song
WHERE song = 'All Hands Against His Own'
"""
try:
    rows = session.execute(query)
    for row in rows:
        print(row.user)
except Exception as e:
    print(e)

Sara Johnson
Jacqueline Lynch
Tegan Levine


### Drop the tables before closing out the sessions

In [20]:
try:
    query = "DROP TABLE IF EXISTS music_history"
    session.execute(query)
    query = "DROP TABLE IF EXISTS music_history_by_session"
    session.execute(query)
    query = "DROP TABLE IF EXISTS users_by_song"
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()